# Desafio NER - Conversão dos Arquivos IOB para JSON e Spacy
* Agrupa por Label ou por tipo de Dataset (Treino, Validação e Teste). 
* Caso deseje agrupar treino com validação (train_dev) basta ajustar o vetor DATASET_TYPE com a constante TRAIN_DEV_DATASET.
* Junção de tokens com espaço em branco.


In [2]:
import os
from pathlib import Path
import re
import string
import json
import random
import sys
import time

from tqdm import tqdm
import spacy
from spacy.tokens import DocBin
from spacy.util import minibatch, compounding
from spacy.training import Example


In [3]:
LABEL_DRUG_PROTEIN = 'DRUG-PROTEIN'
LABEL_CHEMICAL = 'CHEMICAL'
LABEL_DISEASE = 'DISEASE'
LABEL_SPECIES = 'SPECIES'

LABEL_LIST = [LABEL_DRUG_PROTEIN,
              LABEL_CHEMICAL,
              LABEL_DISEASE,    
              LABEL_SPECIES]

LABEL_TO_DIR = {
    LABEL_DRUG_PROTEIN: ['BC2GM', 'JNLPBA'],
    LABEL_CHEMICAL: ['BC4CHEMD','BC5CDR-chem'],
    LABEL_DISEASE: ['BC5CDR-disease', 'NCBI-disease'],    
    LABEL_SPECIES: ['linnaeus', 's800']
}

DATA_ORIGIN_PATH = os.path.join("data","origin")
DATA_PREPARED_PATH = os.path.join("data", "prepared")
DATA_AGGREGATE_PATH = os.path.join(DATA_PREPARED_PATH, "aggregate")

WORD_VECTOR_PATH = "word_vec"
WORD_VECTOR_MODEL_NAME = os.path.join(WORD_VECTOR_PATH,"biomed.model")
WORD_VECTOR_FILE_NAME = os.path.join(WORD_VECTOR_PATH,"biomed_word2vec.txt")
WORD_VECTOR_GLOVE = os.path.join(WORD_VECTOR_PATH,"glove.840B.300d.txt")
WORD_VECTOR_PUBMED_PMC_ORIGINAL = os.path.join(WORD_VECTOR_PATH, "ri-3gram-400-tsv", "vectors.tsv")
VOCAB_PUBMED_PMC = os.path.join(WORD_VECTOR_PATH, "ri-3gram-400-tsv", "vocab.tsv")
WORD_VECTOR_PUBMED_PMC_PREPARED = os.path.join(WORD_VECTOR_PATH, "pubmed_pmc_word2vec.txt")


MODEL_PATH = "model"
MODEL_TRAIN_PATH = os.path.join(MODEL_PATH, "train")
MODEL_ACTUAL_PATH = os.path.join(MODEL_PATH, "actual")

TSV_EXTENSION = ".tsv"
JSON_EXTENSION = ".json"
SPACY_EXTENSION = ".spacy"

AUG_SUFFIXE = "_aug"

TRAIN_DEV_DATASET = "train_dev"
TRAIN_DATASET = "train"
TRAIN_AUGMENT_DATASET = "train_aug"
VALIDATE_DATASET = "devel"
TEST_DATASET = "test"
TRAIN_AUG_VALIDATE_DATASET = "train_dev_aug"

DATASET_TYPE = [TRAIN_DATASET, VALIDATE_DATASET, TEST_DATASET]

### Conversão arquivo IOB para os formatos JSON(formato próprio) e SPACY (para treinamento em linha de comando)
#### Json gerado com formato próprio que será tratado no momento do treinamento

In [4]:
def processa_iob_dataset(lista_annotations, label, dataset_type, dir_dataset, to_lower):
    entidade_atual = ""
    ini_entidade_atual = -1
    entities = []
    sentenca = ""
    
    dataset_iob_file = os.path.join(DATA_ORIGIN_PATH, dir_dataset, dataset_type + TSV_EXTENSION)
    with open(dataset_iob_file) as f_iob:
        for linha in f_iob:
            if len(entidade_atual) > 0 and ("\tO" in linha or "\tB" in linha or linha == "\n"):
                entities.append({"entidade":entidade_atual, 
                                     "start":ini_entity_atual, 
                                     "end": ini_entity_atual + len(entidade_atual),
                                     "label": label
                                    })
                entidade_atual = ""
                ini_entity_atual = -1

            if linha != "\n":
                if (len(sentenca) != 0):
                    sentenca += " "
                if len(entidade_atual) > 0:
                    entidade_atual += " "

                if ("\tO" in linha):
                    linha_tratada = linha.replace("\tO","").replace("\n", "")
                elif("\tB" in linha):
                    ini_entity_atual = len(sentenca)
                    linha_tratada = linha.replace("\tB","").replace("\n", "")
                    entidade_atual = linha_tratada                
                elif("\tI" in linha):
                    linha_tratada = linha.replace("\tI","").replace("\n", "")
                    entidade_atual += linha_tratada
                
                if to_lower:
                    linha_tratada = linha_tratada.to_lower()
                sentenca = sentenca + linha_tratada
            else:
                lista_annotations.append({"texto": sentenca, "entities": entities})
                sentenca = ""
                entities=[]
    return lista_annotations

In [5]:
# path - caminho que será gravado, sem o nome do arquivo
# file_name - nome do arquivo sem extensão
def save_converted_file(lista_annotations, path, file_name, save_json, save_spacy):
    if save_json:
        json_file = os.path.join(path, file_name + JSON_EXTENSION)
        if os.path.exists(json_file):
            os.remove(json_file)
        Path(path).mkdir(parents=True, exist_ok=True)
        with open(json_file, 'w') as json_file:            
            json.dump(lista_annotations, json_file)

    if save_spacy:
        nlp = spacy.blank("en") # load a new spacy model
        db = DocBin() # create a DocBin object
        for an in lista_annotations:
            doc = nlp.make_doc(an['texto']) # create doc object from text
            ents=[]
            for entidade in an['entities']:
                span = doc.char_span(entidade['start'], entidade['end'], label=entidade['label'], alignment_mode="contract")
                if span is None:
                    print ("Span None")
                    print(ner['texto'])
                    print(entidade)
                else:
                    ents.append(span)

            doc.ents = ents
            db.add(doc)

        spacy_file = os.path.join(path, file_name + SPACY_EXTENSION)
        if os.path.exists(spacy_file):
            os.remove(spacy_file)
        db.to_disk(spacy_file) 

In [6]:
def convert_IOB_json_spacy(conv_json=True, conv_spacy=True, group_by_label=True, to_lower=False):
    
    for dataset_type in DATASET_TYPE:
        print(dataset_type)
        if not group_by_label:
            lista_annotations = []
        for label in LABEL_LIST:
            if (group_by_label):
                lista_annotations = []
    
            for dir_dataset in LABEL_TO_DIR[label]:
                lista_annotations = processa_iob_dataset(lista_annotations, label, dataset_type, dir_dataset, to_lower)

            if group_by_label:
                path = os.path.join(DATA_PREPARED_PATH, label)
                file = label + "-" + dataset_type
                save_converted_file(lista_annotations, path, file, save_json=conv_json, save_spacy=conv_spacy)
        if not group_by_label:
            file = dataset_type
            save_converted_file(lista_annotations, DATA_AGGREGATE_PATH, file, save_json=conv_json, save_spacy=conv_spacy)

#### Converte IOB para JSON agrupando por Label

In [7]:
convert_IOB_json_spacy(conv_json=True, conv_spacy=False, group_by_label=True, to_lower=False)

train
devel
test


#### Converte IOB para JSON agrupando por Dataset (Train, Dev e Test)

In [32]:
convert_IOB_json_spacy(conv_json=True, conv_spacy=False, group_by_label=False)

---

### Tratamento de Entidades com Caracteres Especiais

#### Gerar mais exemplos quando existirem caracteres especiais

In [23]:
def trata_sentenca_iob(lista_iob_entries):
    precisa_nova_sentenca = False
    nova_sentenca = []
    pos = 0
    
    token_anterior, tipo_token_anterior = None, None
    
    while pos < len(lista_iob_entries):
        entry = lista_iob_entries[pos]
        entry = entry.replace("\n","")
        split = entry.split("\t")
        token = split[0]
        tipo_token = split[1].strip()
        
        if tipo_token == "O":            
            pos+= 1
            token_anterior, tipo_token_anterior = None, None
            nova_sentenca.append(entry)
            continue
        elif len(token) > 1:
            nova_sentenca.append(entry)
        elif len(token) == 1:
            if token in "-_:/\\" and token_anterior != None and pos < len(lista_iob_entries) - 1: #junta anterior + atual + posterior
                # existe token anterior adiciona, deve ser removido porque será concatenado
                del nova_sentenca[-1]
                
                token_proximo = lista_iob_entries[pos+1].split("\t")[0] 
                tipo_token_proximo = lista_iob_entries[pos+1].split("\t")[1].strip()
                if tipo_token_proximo != "O":
                    precisa_nova_sentenca = True
                    pos+=1
                    novo_token = token_anterior + token + token_proximo
                    nova_sentenca.append(novo_token + "\t" + tipo_token_anterior)                    
                else:
                    nova_sentenca.append(entry)
            
            elif token in ".%+)]}" and token_anterior != None: #junta com o anterior
                precisa_nova_sentenca = True
                # existe token anterior adiciona, deve ser removido porque será concatenado
                del nova_sentenca[-1]
                                
                novo_token = token_anterior + token
                nova_sentenca.append(novo_token + "\t" + tipo_token_anterior)                    
            
            elif token in "([{" and pos < len(lista_iob_entries) - 1 : #junta com o posterior
                
                token_proximo = lista_iob_entries[pos+1].split("\t")[0] 
                tipo_token_proximo = lista_iob_entries[pos+1].split("\t")[1].strip()
                if tipo_token_proximo != "O":
                    precisa_nova_sentenca = True
                    pos+=1
                    novo_token = token + token_proximo
                    nova_sentenca.append(novo_token + "\t" + tipo_token)                    
                else:
                    nova_sentenca.append(entry)
            
            elif token in "\"'*$": #remove o token                
                precisa_nova_sentenca = True                                
            else:
                nova_sentenca.append(entry)
        #if "sulfonatopropoxyl" in " ".join([n for n in nova_sentenca]):
            #print(nova_sentenca)
        # tratamento de erro
        if len(nova_sentenca) == 0:
            print(lista_iob_entries)
            print(token, tipo_token)
        token_anterior = nova_sentenca[-1].split("\t")[0]                 
        tipo_token_anterior = nova_sentenca[-1].split("\t")[1].strip()
        pos+= 1        
        
    if precisa_nova_sentenca:
        return nova_sentenca
    else:
        return None

In [26]:
def augment_iob_file(dir_dataset, label, dataset_type):
    dataset_iob_file = os.path.join(DATA_ORIGIN_PATH, dir_dataset, dataset_type + TSV_EXTENSION)
    lista_novas_sentencas = []
    with open(dataset_iob_file,"r") as f_iob:
        lista_iob_entries =[]
        for linha in f_iob:
            if linha != "\n":
                lista_iob_entries.append(linha)
            else:
                nova_sentenca = trata_sentenca_iob(lista_iob_entries)
                if nova_sentenca != None:
                    lista_novas_sentencas.append(nova_sentenca)
                lista_iob_entries = []
    if len(lista_novas_sentencas) > 0:
        dataset_augment_file = os.path.join(DATA_ORIGIN_PATH, dir_dataset, dataset_type + AUG_SUFFIXE + TSV_EXTENSION)
        with open(dataset_augment_file,"w") as f_aug:
            for nova_sentenca in lista_novas_sentencas:
                for item in nova_sentenca:
                    f_aug.write(item)
                    f_aug.write("\n")
                f_aug.write("\n")

In [28]:
def augment_train_datasets():
    for label in LABEL_LIST:
        for dataset_type in [TRAIN_DATASET, TRAIN_DEV_DATASET]:
            for dir_dataset in LABEL_TO_DIR[label]:
                augment_iob_file(dir_dataset, label, dataset_type)

augment_train_datasets()